In [146]:
from cyvcf2 import VCF
from kipoi_cadd.utils import variant_id_string, dump_to_pickle
from kipoi_cadd.data import KipoiLmdbDataset, KipoiCaddLmdbDataset
import pandas as pd
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [2]:
kipoi_veff_dir = "/data/ouga/home/ag_gagneur/simancas/Projects/kipoi-veff/"
model_dir = kipoi_veff_dir + "tests/models/var_seqlen_model/"
vcf_path = model_dir + "example_files/variants.vcf"
test_dir = "/tmp/kipoi-veff/test_KipoiLmdbDataset/"
lmdb_dir = test_dir + "lmdb"
variant_ids_file = test_dir + "variant_ids.pkl"

## Check functionality of KipoiLmdbDataset

In [3]:
! head -20 {vcf_path}

##fileformat=VCFv4.0
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO
chr22	21541590	rs1111	A	T	.	.	.
chr22	21541952	rs1112	C	C	.	.	.
chr22	30630220	rs1113	T	G	.	.	.
chr22	30630220	rs11122	TAG	G	.	.	.
chr22	30630701	rs1114	A	G	.	.	.
chr22	35503223	rs1115	C	A	.	.	.
chr22	36702137	rs1116	C	A	.	.	.


In [ ]:
"""
var_list = []
for var in VCF(vcf_path):
    # We only support SNV's
    if len(var.REF) == 1:
        var_list.append(variant_id_string(var.CHROM, var.POS, var.REF, var.ALT))
dump_to_pickle(variant_ids_file,pd.Series(var_list))
"""

In [112]:
ds = KipoiLmdbDataset(lmdb_dir, variant_ids_file)

In [99]:
ds.load_all()

,diff:rbp_prb,logitRef:rbp_prb
22:21541590:A:['T'],0.017238,NaN
22:21541952:C:['C'],0.000000,NaN
22:30630220:T:['G'],0.017868,-1.574677
22:30630701:A:['G'],0.032175,NaN
22:35503223:C:['A'],-0.029189,-0.080525
22:36702137:C:['A'],-0.078876,-1.199388


In [109]:
%lprun -f ds.__getitem__ for var in range(6): ds.__getitem__(var)

Timer unit: 1e-06 s

Total time: 0.008897 s
File: /data/ouga04b/ag_gagneur/project_local/kipoi-cadd/kipoi_cadd/data.py
Function: __getitem__ at line 242

Line #      Hits         Time  Per Hit   % Time  Line Contents
   242                                               def __getitem__(self, idx):
   243         6         12.0      2.0      0.1          if self.lmdb_kipoi is None:
   244                                                       self.lmdb_kipoi = lmdb.open(self.lmdb_dir, readonly=True, lock=False)
   245                                                       self.txn = self.lmdb_kipoi.begin()
   246                                           
   247         6         22.0      3.7      0.2          variant_id = self.variant_ids[idx]
   248         6        156.0     26.0      1.8          buf = bytes(self.txn.get(variant_id.encode('ascii')))
   249                                           
   250         6       8371.0   1395.2     94.1          item = pa.deserialize(buf)
   

In [110]:
del ds

## Check functionality of KipoiCaddLmdbDataset

In [167]:
ds_kipoi_cadd = KipoiCaddLmdbDataset([lmdb_dir, lmdb_dir], variant_ids_file)

In [ ]:
ds_kipoi_cadd.load_all()